In [1]:
import pandas as pd

In [2]:
canonical_residues = ['A','C', 'D', 'E', 'F', 'G', 'H', 'I', 'N', 'K', 
    'L', 'M', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

In [3]:
def check_canonical_residues(sequence):
    is_canon=True

    for residue in sequence:
        if residue not in canonical_residues:
            is_canon=False
            break
    
    return is_canon

In [4]:
df_gatsol = pd.read_csv("../../process_data/numerical/GATSOL_export_data.csv")
df_gatsol.head(5)

,id_seq,response,sequence
0,aaeX,0.34,MSLFPVIVVFGLSFPPIFFELLLSLAIFWLVRRVLVPTGIYDFVWH...
1,aas,0.07,MLFSFFRNLCRVLYRVRVTGDTQALKGERVLITPNHVSFIDGILLG...
2,aat,0.08,MRLVQLSRHSIAFPSPEGALREPNGLLALGGDLSPARLLMAYQRGI...
3,abgA,0.31,MESLNQFVNSLAPKLSHWRRDFHHYAESGWVEFRTATLVAEELHQL...
4,abgB,0.49,MQEIYRFIDDAIEADRQRYTDIADQIWDHPETRFEEFWSAEHLASA...


In [5]:
df_protsol = pd.read_csv("../../process_data/numerical/prosol_export_data.csv")
df_protsol.head(5)

,sequence,response
0,MSEMLFFVITTIVKAVVILAVMASLAGLATYAERKVLAYMQRRVGP...,71
1,MSAAIDAVELSRELIRKPSVTPADEGAMDVVERTLAGLGFACRRMR...,21
2,MRMIQRERKREKEEGQLKERTVVNMADPDDNEAEATGLQQYSGETT...,100
3,MSENQDLQKSFLETVKFDDKGLVPAIVQDFETGKVLMMAWMNRQSL...,81
4,MSENLNREPELNIGYITQVIGPVIDAVFSAGQLPKIYNALEVKSKD...,100


In [6]:
df_gatsol["is_in_protsol"] = df_gatsol["sequence"].isin(df_protsol["sequence"].tolist())

In [7]:
df_gatsol["is_in_protsol"].value_counts()

False    1581
True     1206
Name: is_in_protsol, dtype: int64

In [8]:
positive_gatsol = df_gatsol[df_gatsol["is_in_protsol"]==True]
positive_gatsol = positive_gatsol.reset_index()
negative_gatsol = df_gatsol[df_gatsol["is_in_protsol"]==False]
negative_gatsol = negative_gatsol.reset_index()

In [9]:
df_protsol[df_protsol["sequence"] == positive_gatsol["sequence"][5]]

,sequence,response
183404,MLDKIVIANRGEIALRILRACKELGIKTVAVHSSADRDLKHVLLAD...,98


In [10]:
positive_gatsol.iloc[5]

index                                                            8
id_seq                                                        accC
response                                                       0.3
sequence         MLDKIVIANRGEIALRILRACKELGIKTVAVHSSADRDLKHVLLAD...
is_in_protsol                                                 True
Name: 5, dtype: object

In [11]:
length_sequences = []

for index in df_protsol.index:
    length_sequences.append(len(df_protsol["sequence"][index]))

df_protsol["length_sequence"] = length_sequences

In [12]:
df_processed_filter = df_protsol[df_protsol["length_sequence"]<1022]
df_processed_filter.reset_index(inplace=True)
df_processed_filter = df_processed_filter.drop(columns=["index"])
df_processed_filter.shape

(194508, 3)

In [13]:
df_processed_filter["is_canon_seq"] = [check_canonical_residues(sequence) for sequence in df_processed_filter["sequence"]]

In [14]:
df_processed_filter["is_canon_seq"].value_counts()

True    194508
Name: is_canon_seq, dtype: int64

In [15]:
df_processed_filter.to_csv("../../datasets_for_training/numerical_dataset_with_responses.csv", index=False)